# Get New Test Data from Google Finance

In [1]:
import csv
import datetime
import re
import codecs
import requests
import pandas as pd
# import cufflinks as cf
# from plotly.offline import init_notebook_mode, iplot

In [2]:
def get_google_finance_intraday(ticker, period=60, days=1, exchange='NASD'):
    """
    Retrieve intraday stock data from Google Finance.
    
    Parameters
    ----------------
    ticker : str
        Company ticker symbol.
    period : int
        Interval between stock values in seconds.
        i = 60 corresponds to one minute tick data
        i = 86400 corresponds to daily data
    days : int
        Number of days of data to retrieve.
    exchange : str
        Exchange from which the quotes should be fetched
    
    Returns
    ---------------
    df : pandas.DataFrame
        DataFrame containing the opening price, high price, low price,
        closing price, and volume. The index contains the times associated with
        the retrieved price values.
    """
 
    # build url
    url = 'https://finance.google.com/finance/getprices' + \
          '?p={days}d&f=d,o,h,l,c,v&q={ticker}&i={period}&x={exchange}'.format(ticker=ticker, 
                                                                               period=period, 
                                                                               days=days,
                                                                               exchange=exchange)
    
    page = requests.get(url)
    reader = csv.reader(codecs.iterdecode(page.content.splitlines(), "utf-8"))
    columns = ['Open', 'High', 'Low', 'Close', 'Volume']
    rows = []
    times = []
    for row in reader:
        if re.match('^[a\d]', row[0]):
            if row[0].startswith('a'):
                start = datetime.datetime.fromtimestamp(int(row[0][1:]))
                times.append(start)
            else:
                times.append(start+datetime.timedelta(seconds=period*int(row[0])))
            rows.append(map(float, row[1:]))
    if len(rows):
        return pd.DataFrame(rows, index=pd.DatetimeIndex(times, name='Date'), columns=columns)
    else:
        return pd.DataFrame(rows, index=pd.DatetimeIndex(times, name='Date'))

In [4]:
# input data
period = 300
days = 100
exchange = 'NASD'

In [7]:
df_msft = get_google_finance_intraday(ticker='MSFT', period=period, days=days)
df_goog = get_google_finance_intraday(ticker='GOOG', period=period, days=days)
df_aapl = get_google_finance_intraday(ticker='AAPL', period=period, days=days)
df_fb = get_google_finance_intraday(ticker='FB', period=period, days=days)

In [9]:
df_fb.head()

,Open,High,Low,Close,Volume
Date,,,,,
2018-01-16 09:30:00,181.70,181.7000,181.47,181.500,840648.0
2018-01-16 09:35:00,181.27,181.7500,180.80,181.530,1813426.0
2018-01-16 09:40:00,181.01,181.5200,180.86,181.265,1022998.0
2018-01-16 09:45:00,180.90,181.2400,180.80,180.980,896674.0
2018-01-16 09:50:00,181.05,181.1174,180.73,180.900,794518.0


In [11]:
df_fb[['Open', 'High', 'Low', 'Close']].to_csv('data/btc/stock_data_fb.csv')

In [12]:
df_msft[['Open', 'High', 'Low', 'Close']].to_csv('data/btc/stock_data_msft.csv')
df_aapl[['Open', 'High', 'Low', 'Close']].to_csv('data/btc/stock_data_aapl.csv')
df_goog[['Open', 'High', 'Low', 'Close']].to_csv('data/btc/stock_data_goog.csv')

In [17]:
f = 'data/btc/stock_data_msft.csv'
df = pd.read_table(f, sep=',', header=1, index_col=0, names=
                           ['price_open', 'price_high', 'price_low', 'price_close'])

In [19]:
df.head()

,price_open,price_high,price_low,price_close
2018-01-16 09:35:00,90.380,90.46,90.1000,90.370
2018-01-16 09:40:00,90.740,90.75,90.3833,90.390
2018-01-16 09:45:00,90.720,90.79,90.6039,90.735
2018-01-16 09:50:00,90.655,90.78,90.5700,90.720
2018-01-16 09:55:00,90.575,90.72,90.5500,90.660
